In [ ]:
import numpy as np
import os
from skimage import io
import pandas as pd
from COLD import * 
from LBP import * 
from HOG import *

In [ ]:
malesDirectories = ['preprocessedIcdar/Males', 'preprocessedCMP/Males']
femalesDirectories = ['preprocessedIcdar/Females', 'preprocessedCMP/Females']
imgLabel = ['icdar', 'cmp']
featuresTable = []
for i in range(2):
    malesDirectory = malesDirectories[i]
    femalesDirectory = femalesDirectories[i]
    targetDirectories = [malesDirectory, femalesDirectory]
    labels = ['m','f']
    classification = [1,0]
    for j in range(2):
        for imgPath in os.listdir(targetDirectories[j]):
            img = io.imread(targetDirectories[j]+'/'+imgPath)
            imgName = imgPath.split('i')[0]+imgLabel[i]+labels[j]
            coldHist = ColdFeature(img)
            lbpHist = extract_lbp(img)
            hogHist = calculateHOG(img)
            dataValues = [classification[j],imgName]
            dataValues.extend(coldHist)
            dataValues.extend(lbpHist)
            dataValues.extend(hogHist)

            featuresTable.append(dataValues)




COLDLabels = ['COLD_'+str(i) for i in range(100)]
LBPLabels = ['LBP_'+str(i) for i in range(256)]
HOGLabels = ['HOG_'+str(i) for i in range(144)]
labels = ['Gender','ImgName']
labels.extend(COLDLabels)
labels.extend(LBPLabels)
labels.extend(HOGLabels)

featuresTable = np.array(featuresTable)
df = pd.DataFrame(data=featuresTable,columns=labels)

for column in df: 
    if column != 'Gender' and column != 'ImgName':
        df[column] = df[column].astype(float)

#Remove columns where std = 0 
df = df.loc[:, (df != 0).any(axis=0)]

#Non Normalized Dataset
df.to_csv('NonNormalized.csv')

#Calculate standard normalization
for column in df:
    if column != 'Gender' and column != 'ImgName':
        df[column] = (df[column] -
                           df[column].mean()) / df[column].std()

df.to_csv('Dataset.csv')